In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import cv2
import time
import numpy as np

In [2]:
import torch
from torchvision import datasets,transforms,models
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
from torch.optim import Adam,SGD
from torchsummary import summary
import torch.optim.lr_scheduler as lr_scheduler

In [3]:
data = pd.read_csv("kaggle_bee_vs_wasp\labels.csv" ,index_col=False)
data

,id,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
0,1,bee1\10007154554_026417cfd0_n.jpg,1,0,0,0,1,0,0,bee
1,2,bee1\10024864894_6dc54d4b34_n.jpg,1,0,0,0,1,0,1,bee
2,3,bee1\10092043833_7306dfd1f0_n.jpg,1,0,0,0,1,1,0,bee
3,4,bee1\1011948979_fc3637e779_w.jpg,1,0,0,0,1,0,1,bee
4,5,bee1\10128235063_dca17db76c_n.jpg,1,0,0,0,1,0,0,bee
...,...,...,...,...,...,...,...,...,...,...
11416,11417,other_noinsect\581704.jpg,0,0,0,1,0,0,1,other
11417,11418,other_noinsect\581873.jpg,0,0,0,1,0,0,1,other
11418,11419,other_noinsect\581880.jpg,0,0,0,1,0,1,0,other
11419,11420,other_noinsect\6778651038_294d392627_n.jpg,0,0,0,1,0,0,1,other


In [4]:
#change the path in the feature = 'path'
# for i in data.index:
#   data['path'].iloc[i] = data['path'].iloc[i].replace('\\', '/')
# data

In [5]:
#transform the label to class number
labeltool = LabelEncoder()
labeltool.fit(data['label'])
print(labeltool.classes_)
data['label'] = labeltool.transform(data['label'])
data

['bee' 'insect' 'other' 'wasp']


,id,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
0,1,bee1\10007154554_026417cfd0_n.jpg,1,0,0,0,1,0,0,0
1,2,bee1\10024864894_6dc54d4b34_n.jpg,1,0,0,0,1,0,1,0
2,3,bee1\10092043833_7306dfd1f0_n.jpg,1,0,0,0,1,1,0,0
3,4,bee1\1011948979_fc3637e779_w.jpg,1,0,0,0,1,0,1,0
4,5,bee1\10128235063_dca17db76c_n.jpg,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
11416,11417,other_noinsect\581704.jpg,0,0,0,1,0,0,1,2
11417,11418,other_noinsect\581873.jpg,0,0,0,1,0,0,1,2
11418,11419,other_noinsect\581880.jpg,0,0,0,1,0,1,0,2
11419,11420,other_noinsect\6778651038_294d392627_n.jpg,0,0,0,1,0,0,1,2


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11421 entries, 0 to 11420
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   11421 non-null  int64 
 1   path                 11421 non-null  object
 2   is_bee               11421 non-null  int64 
 3   is_wasp              11421 non-null  int64 
 4   is_otherinsect       11421 non-null  int64 
 5   is_other             11421 non-null  int64 
 6   photo_quality        11421 non-null  int64 
 7   is_validation        11421 non-null  int64 
 8   is_final_validation  11421 non-null  int64 
 9   label                11421 non-null  int32 
dtypes: int32(1), int64(8), object(1)
memory usage: 847.8+ KB


In [7]:
#train data
traindf = data[(data['is_validation']==0) & (data['is_final_validation']==0)]
traindf

,id,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
0,1,bee1\10007154554_026417cfd0_n.jpg,1,0,0,0,1,0,0,0
4,5,bee1\10128235063_dca17db76c_n.jpg,1,0,0,0,1,0,0,0
5,6,bee1\10130729734_f68738333e_w.jpg,1,0,0,0,1,0,0,0
6,7,bee1\10166485783_9cd6706c72_n.jpg,1,0,0,0,1,0,0,0
7,8,bee1\10179824_84803290ac_m.jpg,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
11405,11406,other_noinsect\580930.jpg,0,0,0,1,0,0,0,2
11406,11407,other_noinsect\580942.jpg,0,0,0,1,0,0,0,2
11407,11408,other_noinsect\580943.jpg,0,0,0,1,0,0,0,2
11411,11412,other_noinsect\581206.jpg,0,0,0,1,0,0,0,2


In [8]:
#validation data
validationdf = data[data['is_validation']==1]
validationdf

,id,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
2,3,bee1\10092043833_7306dfd1f0_n.jpg,1,0,0,0,1,1,0,0
10,11,bee1\10290912366_20f0dbdb0e_m.jpg,1,0,0,0,1,1,0,0
14,15,bee1\10330536605_77cd1d5e37_n.jpg,1,0,0,0,1,1,0,0
23,24,bee1\10457143913_149e654aaa_n.jpg,1,0,0,0,1,1,0,0
30,31,bee1\10823834_7f6ddb5bce_n.jpg,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
11397,11398,other_noinsect\580382.jpg,0,0,0,1,0,1,0,2
11409,11410,other_noinsect\581089.jpg,0,0,0,1,0,1,0,2
11413,11414,other_noinsect\581451.jpg,0,0,0,1,0,1,0,2
11415,11416,other_noinsect\581668.jpg,0,0,0,1,0,1,0,2


In [9]:
testdf = data[data['is_final_validation']==1]
testdf

,id,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
1,2,bee1\10024864894_6dc54d4b34_n.jpg,1,0,0,0,1,0,1,0
3,4,bee1\1011948979_fc3637e779_w.jpg,1,0,0,0,1,0,1,0
22,23,bee1\1044467943_594f56bd36_n.jpg,1,0,0,0,1,0,1,0
36,37,bee1\1154438668_037d503906_m.jpg,1,0,0,0,1,0,1,0
39,40,bee1\11609248916_80ea630895_n.jpg,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
11412,11413,other_noinsect\581277.jpg,0,0,0,1,0,0,1,2
11414,11415,other_noinsect\581654.jpg,0,0,0,1,0,0,1,2
11416,11417,other_noinsect\581704.jpg,0,0,0,1,0,0,1,2
11417,11418,other_noinsect\581873.jpg,0,0,0,1,0,0,1,2


In [10]:
validationdf['label'] = validationdf['label'].astype(np.int64)
testdf['label']  = testdf['label'] .astype(np.int64)

<ipython-input-10-3f9a6bb38e3d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validationdf['label'] = validationdf['label'].astype(np.int64)
<ipython-input-10-3f9a6bb38e3d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['label']  = testdf['label'] .astype(np.int64)


In [11]:
#definition of trransform method
train_transform = transforms.Compose([transforms.ToPILImage(),
                    transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])

valid_transform = transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])])

In [12]:
#definition of dataset
class BeeDataset():
    #initiate the components 
    def __init__(self, df:pd.DataFrame, imgdir:str, train:bool,
                transforms=None):
        self.df = df
        self.imgdir = imgdir
        self.train = train
        self.transforms = transforms

    def __getitem__(self, index): 
        #join the file path
        im_path = os.path.join(self.imgdir, self.df.iloc[index]["path"])
        x = cv2.imread(im_path)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
        x = cv2.resize(x, (224, 224))

        if self.transforms:
            x = self.transforms(x)

        if self.train:
            y = self.df.iloc[index]["label"]
            return x, y
        else:
            return x

    def __len__(self):
        return len(self.df)

In [13]:
#define a network
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        #load the pretrained models
        #self.model = models.resnet50(pretrained = True)
        #self.model = models.resnet34(pretrained = True)
        #self.model = models.resnet50_bn(pretrained = True)
        #self.model = models.vgg16(pretrained = True)
        #self.model = models.vgg16_bn(pretrained = True)
        #self.model = models.inception_v3(pretrained=True)
        #self.model.aux_logits=False
        #self.model = models.densenet121(pretrained = True)
        #fc_features = self.model.classifier.in_features
        self.model = models.googlenet(pretrained=True)
        self.model.aux_logits=False
        #修改类别为4
        self.model.num_classes = 4
        #self.model.classifier = nn.Linear(fc_features, 4)
        #self.model.classifier._modules['6'] = nn.Sequential(nn.Linear(4096, 4))
    def forward(self,x):
        output = self.model(x)
        return output

In [14]:
#define the dataset
train_dataset = BeeDataset(df =traindf,imgdir = 'kaggle_bee_vs_wasp',train=True,
                          transforms = train_transform)
valid_dataset = BeeDataset(df =validationdf,imgdir = 'kaggle_bee_vs_wasp',train=True,
                          transforms = valid_transform)
test_dataset = BeeDataset(df =testdf,imgdir = 'kaggle_bee_vs_wasp',train=True,
                          transforms = valid_transform)

In [15]:
#define the dataloader
train_loader = DataLoader(dataset = train_dataset,shuffle = True,batch_size=8,num_workers = 0)
valid_loader = DataLoader(dataset = valid_dataset,shuffle = True,batch_size=8,num_workers = 0)
test_loader  = DataLoader(dataset = test_dataset,shuffle = True,batch_size=8,num_workers = 0)

In [16]:
#check whether cuda can be used
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
print(device)

cuda


In [17]:
#define the loss function and the optimization method
criterion = nn.CrossEntropyLoss()

net = Net()
net.to(device)
    
optimizer = torch.optim.SGD(net.parameters(),lr = 0.001,momentum =0.9)
scheduler = lr_scheduler.MultiStepLR(optimizer, [5, 10, 14, 16], gamma=0.5)
#optimizer = optim.Adam(net.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to C:\Users\13746/.cache\torch\hub\checkpoints\googlenet-1378be20.pth


In [18]:
#visualize the network architecture
summary(model = net, input_size = (3, 224, 224),batch_size =8)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [8, 64, 112, 112]           9,408
       BatchNorm2d-2          [8, 64, 112, 112]             128
       BasicConv2d-3          [8, 64, 112, 112]               0
         MaxPool2d-4            [8, 64, 56, 56]               0
            Conv2d-5            [8, 64, 56, 56]           4,096
       BatchNorm2d-6            [8, 64, 56, 56]             128
       BasicConv2d-7            [8, 64, 56, 56]               0
            Conv2d-8           [8, 192, 56, 56]         110,592
       BatchNorm2d-9           [8, 192, 56, 56]             384
      BasicConv2d-10           [8, 192, 56, 56]               0
        MaxPool2d-11           [8, 192, 28, 28]               0
           Conv2d-12            [8, 64, 28, 28]          12,288
      BatchNorm2d-13            [8, 64, 28, 28]             128
      BasicConv2d-14            [8, 64,

In [19]:
trainloss = []
trainacc = []
valloss = []
valacc = []

In [20]:
#define the training function
def train_model(model,optimizer,n_epochs,criterion):
    start_time = time.time()
    for epoch in range(1,n_epochs-1):
        epoch_time = time.time()
        epoch_loss = 0
        correct = 0
        total = 0
        print( "Epoch {}/{}".format(epoch,n_epochs))
    
    #########################train the model
        model.train()
        num_batches = len(train_dataset) // train_loader.batch_size
        for inputs,labels in train_loader:
            
            #get the inputs and labels in training data
            inputs = inputs.to(device)
            #labels = labels.to(device)
            labels = torch.tensor(labels, dtype=torch.long, device=device)

            #zero the parameter gradients
            optimizer.zero_grad()

            #forward + backward + optimize
            output = model(inputs)
            loss = criterion(output,labels)
            loss.backward()
            optimizer.step()

            #compute training loss
            epoch_loss +=loss.item()

            #compute training accuracy
            _,pred =torch.max(output,1)
            correct += (pred.cpu()==labels.cpu()).sum().item()
            total +=labels.shape[0]
        scheduler.step()
        train_loss = epoch_loss/num_batches
        acc = correct/total

    #####################################evaluation     
        model.eval()
        a= 0
        pred_val = 0
        corr = 0
        tot = 0
        num_batches = len(valid_dataset) // valid_loader.batch_size
        
        with torch.no_grad():
            for val_inp,val_label in valid_loader:
                val_inp = val_inp.to(device)
                val_label = val_label.to(device)

                #forward
                out_val = model(val_inp)
                loss = criterion(out_val,val_label)

                #compute evaluation loss
                a += loss.item()

                #compute evaluation accuracy
                _,pred_val = torch.max(out_val,1)
                corr += (pred_val.cpu()==val_label.cpu()).sum().item()
                tot += val_label.shape[0]
            acc_val = corr/tot
            val_loss = a/num_batches

        #print   
        epoch_time2 = time.time()
        trainloss.append(train_loss)
        trainacc.append(acc)
        valloss.append(val_loss)
        valacc.append(acc_val)
        
        print("Duration : {:.4f},Train Loss :{:.4f},Train Acc :{:.4f}, Valid Loss:{:.4f},Valid acc :{:.4f}".format(
        epoch_time2-epoch_time, train_loss, acc, val_loss, acc_val))
    end_time= time.time()
    print("Total time :{:.0f}s".format(end_time - start_time))

In [21]:
#define the test function
def eval_model(model):
    correct = 0
    total = 0
  
    model.eval()
    with torch.no_grad():
        for images,label in test_loader:
            images = images.to(device)
            label = label.to(device)

            #load image to the model
            output = model(images)

            #compute the test accracy
            _, pred = torch.max(output,1)
            correct += (pred == label).sum().item()
            total += label.shape[0]
    print("The accuracy in Test dataset is %.4f %%" %(100*correct/total))

In [22]:
#Train!
train_model(model=net, optimizer=optimizer, n_epochs=20, criterion=criterion)

Epoch 1/20


<ipython-input-20-3408db898a96>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long, device=device)


Duration : 145.8800,Train Loss :0.6518,Train Acc :0.7951, Valid Loss:0.2809,Valid acc :0.9052
Epoch 2/20
Duration : 159.9055,Train Loss :0.3700,Train Acc :0.8679, Valid Loss:0.2087,Valid acc :0.9284
Epoch 3/20
Duration : 153.1555,Train Loss :0.3302,Train Acc :0.8815, Valid Loss:0.2169,Valid acc :0.9244
Epoch 4/20
Duration : 153.3779,Train Loss :0.2961,Train Acc :0.8953, Valid Loss:0.2270,Valid acc :0.9203
Epoch 5/20
Duration : 152.2359,Train Loss :0.2856,Train Acc :0.9006, Valid Loss:0.2603,Valid acc :0.9098
Epoch 6/20
Duration : 150.5595,Train Loss :0.2469,Train Acc :0.9113, Valid Loss:0.2190,Valid acc :0.9284
Epoch 7/20
Duration : 154.8242,Train Loss :0.2171,Train Acc :0.9242, Valid Loss:0.1872,Valid acc :0.9343
Epoch 8/20
Duration : 157.1083,Train Loss :0.2101,Train Acc :0.9261, Valid Loss:0.1869,Valid acc :0.9401
Epoch 9/20
Duration : 151.0022,Train Loss :0.2004,Train Acc :0.9283, Valid Loss:0.1922,Valid acc :0.9314
Epoch 10/20
Duration : 150.3041,Train Loss :0.2001,Train Acc :0.92

In [23]:
eval_model(model=net)

The accuracy in Test dataset is 94.1010 %


In [24]:
trainloss

[0.6517647480396854,
 0.37002348754885456,
 0.330183628049957,
 0.29607007625905213,
 0.28559197601158126,
 0.24689188917240715,
 0.21707214647950981,
 0.21014764636939,
 0.2003820763258714,
 0.2001321838047701,
 0.17999001567213488,
 0.16651543549244618,
 0.16663365909194144,
 0.15892275517610194,
 0.14886116955958348,
 0.14693305888255465,
 0.14903975862586277,
 0.14691435304214998]

In [25]:
trainacc

[0.7950623504219675,
 0.867867489608263,
 0.8814712180375363,
 0.8953268673636479,
 0.900617206197254,
 0.9113238443128857,
 0.9241718100516438,
 0.9260612167779317,
 0.9283285048494773,
 0.9275727421589621,
 0.9350044086156947,
 0.9435697191082,
 0.9420581937271697,
 0.9453331653860688,
 0.9478523743544527,
 0.9526388713943822,
 0.9467187303186799,
 0.9508754251165135]

In [26]:
valloss

[0.2808950195289173,
 0.20870399839357934,
 0.21688821961859536,
 0.22700020505189372,
 0.26030035247530126,
 0.21900703325850246,
 0.1872001313332416,
 0.18691510189526983,
 0.19219793508125244,
 0.2314190980662792,
 0.21009960696133828,
 0.2044139776028536,
 0.2077388029325771,
 0.20850448429020924,
 0.21575977305107027,
 0.21175118789966785,
 0.20671700919658995,
 0.20331756551924082]

In [27]:
valacc

[0.9051774287376382,
 0.9284467713787086,
 0.9243746364165212,
 0.9203025014543339,
 0.9098312972658522,
 0.9284467713787086,
 0.9342641070389761,
 0.9400814426992438,
 0.9313554392088423,
 0.9278650378126818,
 0.9348458406050029,
 0.9331006399069226,
 0.93717277486911,
 0.9307737056428156,
 0.9360093077370564,
 0.9342641070389761,
 0.9360093077370564,
 0.9325189063408958]